<img class="thumb_g_article" data-org-src="https://i.gifer.com/7RXh.gif" data-org-width="680" dmcf-mid="cCD3JaWEk0" dmcf-mtype="image" height="auto" src="https://i.gifer.com/7RXh.gif" width="8000">

# Space Titanic Problem Statement  

# Objective 🚀
In this competition your task is to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly. To help you make these predictions, you're given a set of personal records recovered from the ship's damaged computer system  
 
### File and Data Field Descriptions 

* train.csv - Personal records for about two-thirds (~8700) of the passengers, to be used as training data.
* PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
* HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
* CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
* Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
* Destination - The planet the passenger will be debarking to.
* Age - The age of the passenger.
* VIP - Whether the passenger has paid for special VIP service during the voyage.
* RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
* Name - The first and last names of the passenger.
* Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.
* test.csv - Personal records for the remaining one-third (~4300) of the passengers, to be used as test data.                


The goal is to predict the value of Transported for the passengers in this set.
Final result format:
* PassengerId - Id for each passenger in the test set.
* Transported - The target. For each passenger, predict either True or False.


Allright ! Lets build some models!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics
import missingno as msno
import shap
import gc


from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt 
import seaborn as sns

import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report
from tqdm.notebook import tqdm

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.metrics import accuracy_score, roc_curve,auc, confusion_matrix,precision_recall_curve,precision_recall_curve,plot_precision_recall_curve

import warnings
warnings.simplefilter('ignore')



#plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#selected plotting functions
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

#classes for grid search and cross-validation, function for splitting data and evaluating models
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve

#XGBoost library
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

#Python standard libraries
import time
import warnings

#setting default resolution of figures
mpl.rcParams['figure.dpi'] = 200

# Data Load

In [ ]:
# Loading our training data as train_df and testing data as test_df

train_df =pd.read_csv("../input/spaceship-titanic/train.csv") 
test_df = pd.read_csv("../input/spaceship-titanic/test.csv")

In [ ]:
train_df=train_df.drop('Name', axis=1)
test_df=test_df.drop('Name', axis=1)


In [ ]:
train_df.head()

In [ ]:
#incomplete data analysis
sns.heatmap(train_df.isnull(),yticklabels=False,cbar=False,cmap='viridis')


# EDA

In [ ]:
# Firstly, looking at missing values
#NULL VALUE % IN each column

train_df.isnull().sum()/train_df.count() *100

**We need to fix this by data impuatation methods. Probably may use- SimpleImputer or PyCaret**

In [ ]:
f,ax=plt.subplots(1,2,figsize=(10,3))
train_df['Transported'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Transported')
ax[0].set_ylabel('')
sns.countplot('Transported',data=train_df,ax=ax[1])
ax[1].set_title('Transported')
plt.show()

It is visible that almost half of the passengers were transported. So data in these two classes are balanced.

We will try to check this Transported % using  other features like Cabin, Destination,VIP, Age,etc.

First let us understand the different types of features.

In [ ]:
train_df.groupby(['Destination','Transported'])['Transported'].count()


In [ ]:
# from pycaret.classification import *

In [ ]:
# import pycaret
# print('Using PyCaret Version', pycaret.__version__)
# print('Path to PyCaret: ', pycaret.__file__)

In [ ]:
train_df.groupby('Destination').count()

In [ ]:
train_df.dtypes

In [ ]:
def plot_bar(df, feature, title='', show_percent = False, size=2):
    f, ax = plt.subplots(1,1, figsize=(4*size,3*size))
    total = float(len(df))
    sns.barplot(np.round(df[feature].value_counts().index).astype(int), df[feature].value_counts().values, alpha=0.8, palette='husl')

    plt.title(title)
    if show_percent:
        for p in ax.patches:
            height = p.get_height()
            ax.text(p.get_x()+p.get_width()/2.,
                    height + 3,
                    '{:1.2f}%'.format(100*height/total),
                    ha="center", rotation=45) 
    plt.xlabel(feature, fontsize=12, )
    plt.ylabel('Number of Occurrences', fontsize=12)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
plot_bar(train_df, 'Age',  show_percent=True, size=4)

In [ ]:
# summarize the number of rows with missing values for each column
for i in range(train_df.shape[1]):
	n_miss = train_df.iloc[:,:i].isnull().sum()
	perc = n_miss / train_df.shape[0] * 100
	print('>' , (i, n_miss, perc))

# Preprocessing

### Mean value Imputation for numeric

In [ ]:
#Missing value imputation by basic method, we have some cat var, we will create a better imputation step later
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numeric_col=train_df.select_dtypes(include=numerics).columns
numeric_col
categorical =train_df.select_dtypes(include=[object,bool]).columns
print("numerical col :", numeric_col)
print("categorical col :", categorical)

In [ ]:
#Missing value imputation by basic method, we have some cat var, we will create a better imputation step later
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numeric_col=train_df.select_dtypes(include=numerics).columns
numeric_col
categorical =train_df.select_dtypes(include=[object,bool]).columns
print("numerical col :", numeric_col)
print("categorical col :", categorical)


#applying mean imputation over numeric 
imputer=SimpleImputer(strategy="mean")
imputer.fit(train_df[numeric_col])

train_df[numeric_col]=imputer.transform(train_df[numeric_col])
test_df[numeric_col]=imputer.transform(test_df[numeric_col])


In [ ]:
categorical =train_df.select_dtypes(include=object).columns

train_df[categorical].mode()

In [ ]:
#Getting Dummies for Cat Var
for i in categorical:
    train_df[i]=train_df[i].astype('str')


train_df['Transported']=train_df['Transported'].astype('str')
categorical

In [ ]:
train_df_original=train_df
train_df.head()

In [ ]:
categorical=list(categorical)
categorical.remove('PassengerId') #will encode this later

categorical

In [ ]:
le = LabelEncoder()

def label_encoder(train_df,test_df,categorical):
    for col in categorical:
        train_df[col] = train_df[col].astype(str)
        test_df[col] = test_df[col].astype(str)
        train_df[col] = LabelEncoder().fit_transform(train_df[col])
        test_df[col] =  LabelEncoder().fit_transform(test_df[col])
    return train_df, test_df

train_df ,test_df = label_encoder(train_df,test_df ,categorical)


In [ ]:
train_df["PassengerId"] = le.fit_transform(train_df["PassengerId"])

# Correlation Plot

In [ ]:
f = plt.figure(figsize=(10, 5))
plt.matshow(train_df.corr(), fignum=f.number)
plt.xticks(range(train_df.select_dtypes(['number']).shape[1]), train_df.select_dtypes(['number']).columns, fontsize=14, rotation=45)
plt.yticks(range(train_df.select_dtypes(['number']).shape[1]), train_df.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

Not much of multicollinarity detected..so we are good to proceed.

# Model Creation

In [ ]:

X=train_df.drop('Transported', axis=1)
y=train_df['Transported']

## Test Train Split

In [ ]:
# import xgboost as xgb
# my_model = xgb.XGBClassifier()
# my_model.fit(X_train, y_train)
   
# # Predicting the Test set results
# y_pred = my_model.predict(X_test)
X_train, X_test, y_train, y_test = train_test_split(  X, y, test_size=0.33, random_state=42)

In [ ]:
y_train= le.fit_transform(y_train)
y_test= le.fit_transform(y_test)

# Grid Search

More about hyper-parameter tuning- https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f

In [ ]:


# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5,7]
        }


xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )

# Here we go
random_search.fit(X_train, y_train)


In [ ]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)

In [ ]:
f = plt.figure(figsize=(3, 2))

model = random_search.best_estimator_
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values, X_test, plot_type="bar", plot_size='auto')


# SHAP Beewswarm

The beeswarm plot is designed to display an information-dense summary of how the top features in a dataset impact the model’s output. Each instance the given explanation is represented by a single dot on each feature fow. The x position of the dot is determined by the SHAP value (shap_values.value[instance,feature]) of that feature, and dots “pile up” along each feature row to show density. Color is used to display the original value of a feature (shap_values.data[instance,feature]).


In [ ]:
shap.summary_plot(shap_values, X_test)


In [ ]:
y_pred = random_search.predict_proba(X_test)
y_lst_results=[np.round(x[1]) for x in y_pred]
pd.DataFrame(data={'id':X_test['PassengerId'], 'target':y_lst_results})


In [ ]:
accuracy_score_updated = (y_lst_results == y_test).sum().astype(float) / len(y_lst_results)*100
accuracy_score_updated ## we can definitely imporve on this by feature engineering, nice baseline though

In [ ]:
test_df["PassengerId"]= le.fit_transform(test_df["PassengerId"])

y_pred_validation = random_search.predict_proba(test_df)


In [ ]:
y_lst_results_validation=[np.round(x[1]) for x in y_pred_validation]
test_Df_submission= pd.DataFrame(data={'id':test_df['PassengerId'], 'target':y_lst_results_validation})
y_lst_results_validation=[bool(num) for num in y_lst_results_validation]
test_df["PassengerId"]= le.inverse_transform(test_df["PassengerId"])

# Generating Submission Files

In [ ]:
submission = pd.DataFrame(
    {'PassengerId':test_df["PassengerId"] ,
     'Transported': y_lst_results_validation},columns=['PassengerId', 'Transported'])
submission.to_csv("submission.csv",index=False)


# Please upvote the notebook if you find this helpful ! 🚀

I will be adding more details on this workbook along with some feature engineering + EDA ! Please share your feedback in comments !!